In [4]:
import sys

sys.path.append('/mnt/sdceph/users/nroy/radmc3d-2.0/python/radmc3dPy')
sys.path.append('/mnt/home/nroy/test/chimes-IGNIS/chimes-driver/ext-lib/pfh_python')

#from simpleplot import *
#from radmc3dPy import image
#from gizmopy.load_from_snapshot import load_from_snapshot
#from gizmopy.load_fire_snap import load_fire_snap
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams['axes.facecolor']='w'


params = {"text.color" : 'w',
          "ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
'''
params = {"text.color" : 'k',
          "ytick.color" : "k",
          "xtick.color" : "k",
          "axes.labelcolor" : "k",
          "axes.edgecolor" : "k"}

'''
plt.rcParams.update(params)

In [5]:
#creating line_list, snapshot_list to be looped through and appended to file path to create arrays
line_list = ['CII_158mu', 'CO_J10', 'Halpha', 'NeII_12mu', 'NeIII_15mu', 'NeIII_3869A', 'NeV_14mu', 'NeV_3426A', 'NeVI_7mu', 'NII_6548A', 'OI_145mu', 'OI_6300A', 'OI_63mu', 'OIII_5007A', 'OIII_88mu', 'OIV_25mu', 'SII_6716A', 'SII_6731A', 'SIII_18mu']
snapshot_list = ['151', '152', '155']

In [13]:
file.close()
for s in snapshot_list:
    for l in line_list:
        parent = '/home/jovyan/radmc3d_data/subtract_com_velocity/A4_33000_snum' + s + '/' + l + '/' 
        filepath_tot = parent + 'image_tot.hdf5'
        filepath_con = parent + 'image_continuum.hdf5'
        file_con = h5.File(filepath_con,'r')
        file_tot = h5.File(filepath_tot,'r')
        
        npix_x = np.shape(file_tot['image_array'][:])[0]
        npix_y = np.shape(file_tot['image_array'][:])[1]

        image_array = file_tot['image_array'][:] - file_con['image_array'][:]

        image_array[image_array < 0] = 0

        frequency_array = 3e8 / (file_tot['lambda_array'][:] * 1e-6)

        #moment 0
        integrated_image_array_moment0 = np.zeros((npix_x, npix_y))
        delta_nu = np.abs(frequency_array[1:] - frequency_array[:-1])
        for i in range(npix_x):
            for j in range(npix_y):
                integrated_image_array_moment0[i, j] = np.sum(delta_nu * (image_array[i, j, :-1] + image_array[i, j, 1:]) / 2.0)

        integrated_image_array_moment0[0,0] = 0 #final moment 0 array
        moment0_file_array = integrated_image_array_moment0 #final moment 0 array - to go in hdf5
        
        #moment 1
        N = np.size(frequency_array)
        velocity_arr = np.linspace(-1000, 1000, N)

        moment1_noint = velocity_arr * image_array

        integrated_image_array_moment1 = np.zeros((npix_x, npix_y))

        for i in range(npix_x):
            for j in range(npix_y):
                integrated_image_array_moment1[i, j] = np.sum(delta_nu * (moment1_noint[i, j, :-1] + moment1_noint[i, j, 1:]) / 2.0)

        integrated_image_array_moment1[0,0] = 0

        isolate_moment_1 = integrated_image_array_moment1 / integrated_image_array_moment0
        isolate_moment_1[0,0] = 0 #final moment 1 array
        moment1_file_array = isolate_moment_1 #final moment 1 array - to go in hdf5
        
        #moment 2
        isolate_moment_1_3D = np.empty([len(image_array), len(image_array), len(velocity_arr)])

        for i in range(len(velocity_arr)):
            for j in range(len(image_array)):
                for k in range(len(image_array)):
                    isolate_moment_1_3D[j][k][i] = isolate_moment_1[j][k]

        moment2_noint = image_array * ((velocity_arr - isolate_moment_1_3D) * (velocity_arr - isolate_moment_1_3D))

        integrated_image_array_moment2 = np.zeros((npix_x, npix_y))

        for i in range(npix_x):
            for j in range(npix_y):
                integrated_image_array_moment2[i, j] = np.sum(delta_nu * (moment2_noint[i, j, :-1] + moment2_noint[i, j, 1:]) / 2.0)

        integrated_image_array_moment2[0,0] = 0

        isolate_moment_2 = integrated_image_array_moment2 / integrated_image_array_moment0
        isolate_moment_2[0,0] = 0

        isolate_moment_2 = np.sqrt(isolate_moment_2) #final moment 2 array
        moment2_file_array = isolate_moment_2 #final moment 2 array - to go in hdf5
        
        #creates moment dictionary for 0, 1, 2 for the specific snapshot/line
        moments_dict = {'M0': moment0_file_array, 'M1': moment1_file_array, 'M2': moment2_file_array}
        
        MOMENTS = s + "_moments_all_lines.hdf5"
        
        #saves to hdf5 file (one for each snapshot)
        file = h5.File(MOMENTS, 'w')
        for key in moments_dict.keys():
            file.create_dataset(key, data = moments_dict[key])
        file.close()

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
'''
DATA SAVING IDEAS // IGNORE

Candy = {"sweetness": 1}
jonny = {"holding": Candy}
print(jonny["holding"["sweetness"]])

        
        # Open HDF5 file and write in the data_dict structure and info
        f = h5py.File('test.hdf5', 'w')
        for grp_name in data_dict:
            grp = f.create_group(grp_name)
            for dset_name in data_dict[grp_name]:
                dset = grp.create_dataset(dset_name, data = data_dict[grp_name][dset_name])
                print(grp_name, dset_name, data_dict[grp_name][dset_name])
        f.close()
        
        for i in range(np.size(A)):
            print(A[i])
      
data_file = h5py.File('file.hdf5', 'w')
data_file.create_dataset('group_name', data=data_matrix)
data_file.close()
      
        dict_test = {'a': np.ones((100,100)), 'b': np.zeros((100,100))}
        hf = h5py.File('dict_data.h5', 'w')
        dict_group = hf.create_group('dict_data')
        for k, v in dict_test.items():
            dict_group[k] = v
        hf.close()
'''